######  fMRI Decoding Project Showcase

 


In [1]:
from nilearn import datasets, image, plotting
from nilearn.input_data import NiftiMasker
from nilearn.image.image import mean_img
from nilearn.image import index_img
import pandas as pd


First subject functional nifti images (4D) are at: C:\Users\Ziqi/nilearn_data\haxby2001\subj1\bold.nii.gz
Number of trials:  324
C:\Users\Ziqi/nilearn_data\haxby2001\mask.nii.gz


Preprocessing the Haxby Dataset


In [2]:
#import Haxby et al.(2001): Faces and Objects in Ventral Temporal Cortex (fMRI)
# Subjects 5 and 6 don't have complete label or anatomical information, only included subjects 1-4
haxby_dataset = datasets.fetch_haxby(subjects=4)

#load nifti images for the given subjects. Range 0-3
#defaults to subject 2
def loadSubject(subjectNum = 1):
    # 'func' is a list of filenames: one for each subject
    fmri_filename = haxby_dataset.func[subjectNum]
    # print basic information on the dataset
    print('First subject functional nifti images (4D) are at: %s' %
          fmri_filename)  # 4D data
    return fmri_filename

#plotting subject's anatomical brain
def plotAnat(subjectNum = 1):
    path = haxby_dataset.anat[subjectNum]
    plotting.plot_stat_map(path, threshold=3)
    plotting.show()

#plotting mean functionam MRI
def plotMeanFunc(subjectNum = 1):
    mean_haxby = mean_img(haxby_dataset.func[subjectNum])
    plotting.plot_stat_map(mean_haxby, threshold=3)
    plotting.show()

#plotting one random scan of fMRI
def plotRandomFunc(subjectNum = 1):
    rand_func = index_img(haxby_dataset.func[subjectNum], 30)
    plotting.plot_stat_map(rand_func, threshold=3)
    plotting.show()

fmri_filename = loadSubject(0)
# plotAnat(subjectNum = 2)
# plotMeanFunc(2)
# plotRandomFunc(2)

behavioral = pd.read_csv(haxby_dataset.session_target[0], sep=" ")
conditions = behavioral['labels']

facecat_mask = conditions.isin(['face', 'cat'])
conditions_facecat = conditions[facecat_mask]
session_facecat = behavioral[facecat_mask].to_records(index = False)

facehouse_mask = conditions.isin(['face', 'house'])
conditions_facehouse = conditions[facehouse_mask]
session_facehouse = behavioral[facehouse_mask].to_records(index = False)

threeway_mask = conditions.isin(['face', 'house', 'cat'])
conditions_threeway = conditions[threeway_mask]
session_threeway = behavioral[threeway_mask].to_records(index = False)
print("Number of trials: ", len(conditions_threeway))
mask_filename = haxby_dataset.mask
#masking the data from 4D image to 2D array: voxel x time
#with smothing and standardization
masker = NiftiMasker(mask_img=mask_filename, smoothing_fwhm=4, standardize=True, memory="nilearn_cache", memory_level=1)
print(haxby_dataset.mask)
X = masker.fit_transform(fmri_filename)

# Apply our condition_mask
FC = X[facecat_mask]

FH = X[facehouse_mask]

FHC = X[threeway_mask]
#three-way classification with NN
FHC_train = FHC[:250]
conditions_train = conditions_threeway[1:250]
FHC_val = FHC[250:]
Y_val = conditions_threeway[250:]



First subject functional nifti images (4D) are at: C:\Users\Ziqi/nilearn_data\haxby2001\subj1\bold.nii.gz
Number of trials:  324
C:\Users\Ziqi/nilearn_data\haxby2001\mask.nii.gz


In [3]:
from loadingData import *
from sklearn.svm import SVC
from sklearn.feature_selection import SelectPercentile, f_classif, SelectKBest
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from nilearn import image
from nilearn.plotting import plot_stat_map, show
from sklearn.model_selection import LeaveOneGroupOut, cross_val_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier

# Define the dimension reduction to be used.
# Here we use a classical univariate feature selection based on F-test,
# namely Anova. When doing full-brain analysis, it is better to use
# SelectPercentile, keeping 5% of voxels
# (because it is independent of the resolution of the data).
feature_selection = SelectPercentile(f_classif, percentile=5)

#one-vs-the-rest
#cited from https://scikit-learn.org/stable/modules/svm.html#multi-class-classification
lin_svc = LinearSVC()
facecathouse_svc = Pipeline([('anova', feature_selection), ('svc', lin_svc)])
facecathouse_svc.fit(FHC, conditions_threeway)

another_svc = OneVsRestClassifier(Pipeline([('anova', SelectKBest(f_classif, k=500)), ('svc', SVC(kernel = 'linear'))]))
another_svc.fit(FHC, conditions_threeway)

# Output accuracy
# Define the cross-validation scheme used for validation.
# Here we use a LeaveOneGroupOut cross-validation on the session group
# which corresponds to a leave-one-session-out
def modelAccuracy(model, X, conditions, groups):
    cv = LeaveOneGroupOut()

    # Compute the prediction accuracy for the different folds (i.e. session)
    cv_scores = cross_val_score(model, X, conditions, cv=cv, groups=groups)

    # Return the corresponding mean prediction accuracy
    classification_accuracy = cv_scores.mean()

    # Print the results
    print("Classification accuracy: %.4f / Chance level: %f" %
          (classification_accuracy, 1. / len(conditions.unique())))

print("Linear model on face vs cat vs house: ")
modelAccuracy(facecathouse_svc, FHC, conditions_threeway, session_threeway)

print("The second model on face vs cat vs house: ")
modelAccuracy(another_svc, FHC, conditions_threeway, session_threeway)

cross_validation = cross_val_score(facecathouse_svc, FHC, conditions_threeway, cv = 5, verbose = 1)
print("Linear kernel model cross validation score: ", cross_validation.mean())


ModuleNotFoundError: No module named 'loadingData'

Using an NN model

In [4]:
import numpy as np
from loadingData import *
from keras import models
from keras.layers import Dense
from sklearn.preprocessing import OneHotEncoder, StandardScaler

#need to one-hot encode the Y labels
enc = OneHotEncoder()
#cited from https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/
Y = enc.fit_transform(conditions_threeway[:, np.newaxis]).toarray()
labels_train = Y[:250]
labels_val = Y[250:]


# experimental attempt to transform the 2D voxel*time array
reshaped_X = np.empty(shape = (324, 798))
reshaped_dimension = np.empty(shape = (1, 798))
subject = []
dim = []
# for dimension in range(0, 324):
#     for x in range(0, 797):
#         sum_series = 0
#         for i in range(0, 49):
#             index = x * 50 + i
#             sum_series += FHC[dimension][index]
#         dim.append(sum_series/49)
#     subject.append(dim)
#
# reshaped_X = np.array(subject)

for i in range(0, 324):
    old_dim = FHC[i]
    new_dim = np.mean(old_dim[:(len(old_dim)// 50) * 50].reshape(-1, 50), axis=1)
    subject.append(new_dim)

reshaped_X = np.array(subject)
print(reshaped_X.shape)
print(reshaped_X)

reshaped_Xtrain = reshaped_X[:250]
reshaped_Xval = reshaped_X[250:]

# model = models.Sequential()
# # model.add(Dense(64, input_dim = 39912, activation='relu'))
# # model.add(Dense(32, input_dim = 64, activation='relu'))
# model.add(Dense(16, input_dim = 39912, activation='relu'))
# # model.add(Dense(8, input_dim=16, activation='relu'))
# model.add(Dense(3, activation='softmax'))
# model.summary()
#
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.fit(FHC_train, labels_train, batch_size=5, epochs=50, verbose=1)
# score = model.evaluate(FHC_val, labels_val)
#
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

smaller_model = models.Sequential()
# smaller_model.add(Dense(64, input_dim = 798, activation='relu'))
# smaller_model.add(Dense(32, input_dim = 64, activation='relu'))
smaller_model.add(Dense(16, input_dim = 798, activation='relu'))
smaller_model.add(Dense(8, input_dim=16, activation='relu'))
smaller_model.add(Dense(3, activation='softmax'))
smaller_model.summary()

smaller_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
smaller_model.fit(reshaped_Xtrain, labels_train, batch_size=20, epochs=50, verbose=1)
score = smaller_model.evaluate(reshaped_Xval, labels_val)

print('Model with reduced input: ')
print('Test loss:', score[0])
print('Test accuracy:', score[1])


ModuleNotFoundError: No module named 'loadingData'